In [1]:
#导入一系列包
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
file_path2='D:/files/data/stopwordlisteng.TXT'
stopworddic1=stop_words
data=[]
with open(file_path2, "r") as f:
    for line in f.readlines():
        line = line.strip('\n')  #去掉列表中每一个元素的换行符
        data.append(line)
        #print(line)
for i in range(len(data)):
    stopworddic1.append(data[i])

fh=[',','(',')','e','.','g','-']

for i in range(1000):
    fh.append(str(i))

for i in range(len(fh)):
    stopworddic1.append(fh[i])

In [4]:
# Import Dataset
file_path='D:/files/data/finaldata/WOS.xls'
df=pd.read_excel(file_path)
data = df.keycontent.values.tolist()

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

#print(data_words[:1])

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod)

In [7]:
stop_words = stopworddic1
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[:1])

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 2), (8, 2), (9, 3), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 2), (19, 1), (20, 3), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 6), (31, 1), (32, 1), (33, 1), (34, 3), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 4), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 3), (51, 1), (52, 1), (53, 4), (54, 1), (55, 1), (56, 1), (57, 1), (58, 2), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 4), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 9), (73, 1)]]


In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.5,
                                           per_word_topics=True)

In [11]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.100597 -0.136564       1        1  22.384225
1      0.103020  0.142774       2        1  18.216644
8      0.123197 -0.029453       3        1  14.942548
4      0.022823  0.143868       4        1  10.395680
7      0.100720  0.060125       5        1   8.185393
3     -0.102363 -0.157987       6        1   7.981752
2      0.020686  0.172127       7        1   7.447410
6      0.014113 -0.254509       8        1   5.466453
5     -0.382794  0.059620       9        1   4.979894, topic_info=               Term          Freq         Total Category  logprob  loglift
term                                                                      
423         traffic   2467.000000   2467.000000  Default  30.0000  30.0000
120           model   3873.000000   3873.000000  Default  29.0000  29.0000
212         control   4872.000000   4872.000000  Default  28.0000  28.0000
6321            cav   1253.000000   1253.000000  Default  27.0000  27.0000
739          driver   2430.000000   2430.000000  Default  26.0000  26.0000
65           system   4608.000000   4608.000000  Default  25.0000  25.0000
6322           cavs   1189.000000   1189.000000  Default  24.0000  24.0000
92            datum   1534.000000   1534.000000  Default  23.0000  23.0000
258         network   1036.000000   1036.000000  Default  22.0000  22.0000
427             car   2535.000000   2535.000000  Default  21.0000  21.0000
369        automate   2480.000000   2480.000000  Default  20.0000  20.0000
72          vehicle  14011.000000  14011.000000  Default  19.0000  19.0000
308   communication    981.000000    981.000000  Default  18.0000  18.0000
327            time   2732.000000   2732.000000  Default  17.0000  17.0000
226            road   1968.000000   1968.000000  Default  16.0000  16.0000
745    intersection    968.000000    968.000000  Default  15.0000  15.0000
36           method   2076.000000   2076.000000  Default  14.0000  14.0000
422          safety   1948.000000   1948.000000  Default  13.0000  13.0000
263     performance   1337.000000   1337.000000  Default  12.0000  12.0000
59           sensor   1270.000000   1270.000000  Default  11.0000  11.0000
346           drive   2719.000000   2719.000000  Default  10.0000  10.0000
433         parking    880.000000    880.000000  Default   9.0000   9.0000
8        autonomous   8391.000000   8391.000000  Default   8.0000   8.0000
986         connect    814.000000    814.000000  Default   7.0000   7.0000
244      controller   1184.000000   1184.000000  Default   6.0000   6.0000
223            path   1171.000000   1171.000000  Default   5.0000   5.0000
24      environment   1648.000000   1648.000000  Default   4.0000   4.0000
147           study   2417.000000   2417.000000  Default   3.0000   3.0000
303      trajectory   1077.000000   1077.000000  Default   2.0000   2.0000
1898            avs    700.000000    700.000000  Default   1.0000   1.0000
...             ...           ...           ...      ...      ...      ...
356         message     75.484988     76.330259   Topic9  -5.4478   2.9886
572          expert     74.994949     75.840549   Topic9  -5.4543   2.9885
1086   distribution     75.408764     76.289493   Topic9  -5.4488   2.9881
3652           deep     70.405061     71.250724   Topic9  -5.5175   2.9878
564             pre     66.321362     67.167024   Topic9  -5.5772   2.9871
1034   additionally     63.763680     64.609491   Topic9  -5.6166   2.9866
2531   intelligence     59.157003     60.002726   Topic9  -5.6915   2.9856
1216  heterogeneous     58.019621     58.865507   Topic9  -5.7110   2.9853
1196           load     52.530402     53.376132   Topic9  -5.8103   2.9838
258         network    973.785630   1036.894289   Topic9  -2.8906   2.9370
255           learn    279.692940    310.984599   Topic9  -4.1380   2.8937
579       reduction    219.303931    249.826260   Topic9  -4.38

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.047*"vehicle" + 0.031*"driver" + 0.026*"automate" + 0.022*"drive" + '
  '0.018*"study" + 0.017*"autonomous" + 0.016*"car" + 0.014*"safety" + '
  '0.013*"human" + 0.012*"technology"'),
 (1,
  '0.062*"control" + 0.057*"vehicle" + 0.026*"autonomous" + 0.024*"model" + '
  '0.022*"base" + 0.019*"propose" + 0.018*"controller" + 0.018*"path" + '
  '0.017*"trajectory" + 0.015*"method"'),
 (2,
  '0.071*"model" + 0.021*"vehicle" + 0.019*"equilibrium" + 0.018*"time" + '
  '0.017*"platoon" + 0.016*"dynamic" + 0.014*"strategy" + 0.013*"correlation" '
  '+ 0.012*"derive" + 0.011*"propose"'),
 (3,
  '0.031*"parking" + 0.025*"cost" + 0.025*"travel" + 0.025*"avs" + '
  '0.023*"pattern" + 0.022*"system" + 0.021*"commute" + 0.020*"vehicle" + '
  '0.020*"study" + 0.019*"autonomous"'),
 (4,
  '0.032*"car" + 0.028*"sensor" + 0.027*"method" + 0.025*"base" + '
  '0.024*"autonomous" + 0.020*"propose" + 0.020*"datum" + 0.018*"detection" + '
  '0.018*"vehicle" + 0.017*"drive"'),
 (5,
  '0.056*"network"

In [14]:
from pandas.core.frame import DataFrame
texts1=DataFrame(texts)
topics = [lda_model[corpus[i]] for i in range(len(texts1))]

In [26]:
len(topics)


3904

In [32]:
topics[3903:3904]

[([(0, 0.68189),
   (1, 0.05214099),
   (2, 0.028223984),
   (3, 0.011355651),
   (5, 0.18130614),
   (6, 0.016441213),
   (8, 0.022538627)],
  [(9, [5, 1, 0, 8, 2, 3]),
   (65, [0, 8, 1, 3, 6]),
   (67, [0, 1, 8]),
   (72, [0, 1, 8, 6, 2, 5, 3, 7]),
   (80, [0, 5]),
   (91, [0, 8, 1]),
   (92, [5, 0]),
   (94, [0, 1, 8]),
   (95, [0, 8, 1, 5, 6, 3]),
   (106, [5, 1, 2, 3]),
   (114, [0, 5, 6]),
   (120, [2, 1, 5, 0, 3]),
   (121, [0, 6]),
   (135, [0, 5, 8, 6]),
   (137, [0]),
   (147, [0, 3, 2, 5, 1]),
   (156, [0]),
   (171, [0, 1]),
   (186, [5, 0, 6]),
   (212, [1, 0, 6, 7]),
   (216, [0, 5, 2, 8]),
   (257, [1]),
   (263, [5, 1]),
   (327, [5, 0, 2, 1, 3, 8]),
   (330, [0, 5, 1, 6]),
   (339, [0, 1]),
   (346, [0, 1]),
   (361, [0, 6]),
   (369, [0]),
   (419, [5, 0, 1, 3]),
   (429, [0, 1]),
   (460, [0, 6]),
   (560, [1, 2]),
   (611, [0, 1, 8]),
   (613, [0, 8, 1]),
   (629, [5, 0, 1, 8]),
   (637, [0, 5, 2, 1]),
   (687, [0, 8, 1, 6]),
   (701, [0, 1]),
   (714, [0, 8, 1]),
 

In [19]:
j=0
def topics_document_to_dataframe(topics_document, num_topics):
    global j
    j=j+1
    res = pd.DataFrame(columns=range(num_topics))
    test_2 = pd.DataFrame()  
    for i in range(9):
        test_2.loc[str(j),i]=0
    res1=test_2
    #res.index=0
    #res.ix[]
    for topic_weight in topics_document:
        #print(topic_weight)
        #print(topic_weight[1])
       # print(res)
        #print(res.ix[0, topic_weight[0]])
        res1.loc[str(j),topic_weight[0]] = topic_weight[1]
    return res1

In [34]:
document_topic=[]
document_topic = pd.concat([topics_document_to_dataframe(topics_document[0], 9) for topics_document in topics])

In [21]:
#document_topic

0         1         2         3         4         5         6  \
1   0.031502  0.370239  0.000000  0.010232  0.000000  0.000000  0.042091   
2   0.241201  0.060621  0.076970  0.037276  0.063103  0.025543  0.379131   
3   0.023311  0.108878  0.417131  0.018500  0.221563  0.015925  0.033831   
4   0.000000  0.736556  0.104885  0.021498  0.031290  0.000000  0.000000   
5   0.000000  0.702839  0.010293  0.021033  0.000000  0.123051  0.048123   
6   0.056612  0.700110  0.106525  0.000000  0.000000  0.000000  0.022012   
7   0.000000  0.321490  0.116163  0.000000  0.000000  0.000000  0.372273   
8   0.056090  0.328029  0.045320  0.000000  0.092926  0.000000  0.074639   
9   0.000000  0.345636  0.015549  0.000000  0.000000  0.168032  0.137369   
10  0.048662  0.636526  0.052798  0.023737  0.090408  0.000000  0.054523   

           7         8  
1   0.000000  0.521988  
2   0.000000  0.111601  
3   0.000000  0.154907  
4   0.052380  0.031288  
5   0.000000  0.068155  
6   0.051970  0.042316  
7   0.027912  0.132708  
8   0.000000  0.381782  
9   0.000000  0.304258  
10  0.038969  0.049352

In [22]:
#type(document_topic)

pandas.core.frame.DataFrame

In [35]:
document_topic.to_excel('D:/files/data/finaldata/wosdataframe.xls', index=False)

In [37]:
a=document_topic.sort_values(0, ascending=False)[0].head(20)

In [38]:
type(a)

pandas.core.series.Series

In [39]:
a[0]

0.8757424354553223

In [10]:
#b=[]


In [11]:

#for i in range(2,51):
#    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                           id2word=id2word,
#                                           num_topics=i, 
#                                           random_state=100,
#                                           update_every=1,
#                                           chunksize=100,
#                                           passes=10,
#                                           alpha=0.5,
#                                           per_word_topics=True)
    # Compute Perplexity
    #print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

    # Compute Coherence Score
#    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
#    coherence_lda = coherence_model_lda.get_coherence()
#    print('\nCoherence Score: ', coherence_lda)
#    b.append(coherence_lda)


Coherence Score:  0.351474705184158

Coherence Score:  0.40530306894321594

Coherence Score:  0.41579457860025426

Coherence Score:  0.4244943451484521

Coherence Score:  0.44592991622425965

Coherence Score:  0.41716709434972793

Coherence Score:  0.40542448800011444

Coherence Score:  0.39496719470824004

Coherence Score:  0.3978001096033639

Coherence Score:  0.3912793563273224

Coherence Score:  0.41102630942344004

Coherence Score:  0.3903474960072624

Coherence Score:  0.3755123538444649

Coherence Score:  0.4143424989424138

Coherence Score:  0.3847436726679617

Coherence Score:  0.37536799341289384

Coherence Score:  0.3706289375659481

Coherence Score:  0.37971223778533264

Coherence Score:  0.370206354048918

Coherence Score:  0.35591176620135756

Coherence Score:  0.35647118907414654

Coherence Score:  0.36022653584753195

Coherence Score:  0.35246129522411357

Coherence Score:  0.33943880527979603

Coherence Score:  0.3611656067160172

Coherence Score:  0.3597499328898402
